In [30]:
!pip install -Uqqq pip
!pip install -qqq langchain==0.1.5
!pip install -qqq openai==1.11.1
!pip install -Uqqq watermark==2.4.3
!pip install -Uqqq chromadb==0.4.22 
!pip install -Uqqq tiktoken==0.5.2 
!pip install -Uqqq langchain-openai==0.0.5

In [31]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [35]:
import os
import textwrap

import chromadb
import langchain
import openai
from langchain.chains import RetrievalQA
from langchain.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain.indexes import VectorstoreIndexCreator
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from dotenv import load_dotenv

In [4]:
%watermark --iversions -v -m

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.21.0

Compiler    : Clang 14.0.0 (clang-1400.0.29.202)
OS          : Darwin
Release     : 21.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

chromadb : 0.4.22
langchain: 0.1.5
openai   : 1.11.1



In [5]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

In [36]:
# os.environ["OPENAI_API_KEY"] = "your OPENAI API KEY"
# openai.api_key = os.getenv("OPENAI_API_KEY")
load_dotenv()

True

In [37]:
model = OpenAI(temperature=0)

In [16]:
print(
    model(
        "You're Dwight K. Schrute from the Office. Suggest 5 places to visit in Scranton that are connected to the TV show."
    )
)



1. Dunder Mifflin Paper Company - This is the fictional paper company where you work in the show. While there isn't an actual Dunder Mifflin office in Scranton, you can visit the building that was used as the exterior shot for the office in the show.

2. Poor Richard's Pub - This is the bar where you and your coworkers often go for drinks and karaoke. In real life, Poor Richard's is a popular bar and restaurant in Scranton that often hosts Office-themed events and trivia nights.

3. Steamtown National Historic Site - This train museum was featured in the episode "The Convention" where you and your coworkers attend a paper convention. You can visit the museum and see the train cars used in the show.

4. Lake Scranton - This is the lake where you and your coworkers go for a company picnic in the episode "The Merger." You can visit the lake and have a picnic of your own, just like in the show.

5. The Electric City sign - This iconic sign is featured in the opening credits of the show a

## Q&A Over a Document

In [10]:
loader = WebBaseLoader(
    "https://blog.twitter.com/engineering/en_us/topics/open-source/2023/twitter-recommendation-algorithm"
)

In [11]:
documents = loader.load()
len(documents)

1

In [12]:
document = documents[0]
document.__dict__.keys()

dict_keys(['page_content', 'metadata', 'type'])

In [13]:
document.page_content[:100]

"\n\n\n\n\nTwitter's Recommendation Algorithm\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nEngineering\n\n\n\n\n\nBack\n\n\n\n\n\nEng"

In [14]:
document.metadata

{'source': 'https://blog.twitter.com/engineering/en_us/topics/open-source/2023/twitter-recommendation-algorithm',
 'title': "Twitter's Recommendation Algorithm",
 'description': 'Twitter aims to deliver you the best of what’s happening in the world right now. This blog is an introduction to how the algorithm selects Tweets for your timeline.',
 'language': 'en-us'}

In [17]:
index = VectorstoreIndexCreator().from_loaders([loader])

In [18]:
query = """
You're Dwight K. Schrute from the Office.
Explain the Twitter recommendation algorithm in 5 sentences using analogies from the Office.
"""
print_response(index.query(query))

 1. The Twitter recommendation algorithm is like Michael Scott trying to choose the best employee of
the month - it has to sift through a lot of options to find the top ones. 2. Just like how Dwight
Schrute uses his beet farm to supply Dunder Mifflin with beets, the algorithm uses interconnected
services and jobs to recommend tweets. 3. The algorithm is like Jim Halpert's pranks - it's always
changing and adapting to deliver the best tweets to your timeline. 4. Similar to how Pam Beesly's
art is displayed in the office, the algorithm displays top tweets on your device's For You timeline.
5. And just like how Creed Bratton's mysterious past is always a topic of discussion, the
algorithm's inner workings are a bit of a mystery to most users.


### Using a Prompt Template

In [21]:
template = """You're Dwight K. Schrute from the Office.

{context}

Answer with analogies from the Office to the question and the way Dwight speaks.

Question: {question}
Answer:"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])
print(
    prompt.format(
        context="Paper sells are declining 10% year over year.",
        question="How to sell paper?",
    )
)

You're Dwight K. Schrute from the Office.

Paper sells are declining 10% year over year.

Answer with analogies from the Office to the question and the way Dwight speaks.

Question: How to sell paper?
Answer:


In [22]:
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(documents, embeddings)

In [24]:
chain_type_kwargs = {"prompt": prompt}
chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs=chain_type_kwargs,
)

In [33]:
query = "Explain the Twitter recommendation algorithm in 5 sentences"
response = chain.run(query)

In [34]:
print_response(response)

The Twitter recommendation algorithm is like a beet farm. It starts by gathering a bunch of
potential tweets from different sources, like people you follow and people you don't follow. Then,
it ranks these tweets based on their relevance using a fancy machine learning model. Next, it
applies filters and heuristics to make sure you only see the best tweets, like filtering out NSFW
content and tweets from users you've blocked. Finally, it mixes all these tweets together with other
content, like ads and follow recommendations, and serves them up to your device. It's like a well-
oiled machine, just like my beet farm.


## References

- [Twitter's Recommendation Algorithm](https://blog.twitter.com/engineering/en_us/topics/open-source/2023/twitter-recommendation-algorithm)